In [ ]:
import numpy as np
import pandas as pd
import cv2
import os
import collections
import matplotlib as mpl
from tqdm import tqdm
from typing import List, Tuple, Dict, Any
import gc

# Get statistics for Fundamental Matrix Estimation using Mutal Nearest Neighbour Matching (NN2W)

In [ ]:
#####################
### FUNCTIONS
#####################

def normalize_descriptors(desc:np.array) -> np.array:
    """Creates unit vectors for each descriptor."""    
    _n = np.linalg.norm(desc, axis=1, ord=2) # Get norms of each vector
    _d = desc / _n.reshape(-1, 1)            # Build unit vector
    
    return _d

def nn_match_two_way(desc1, desc2, nn_thresh):
    """
    Performs two-way nearest neighbor matching of two sets of descriptors, such
    that the NN match from descriptor A->B must equal the NN match from B->A.

    Inputs:
      desc1 - NxM numpy matrix of N corresponding M-dimensional descriptors.
      desc2 - NxM numpy matrix of N corresponding M-dimensional descriptors.
      nn_thresh - Optional descriptor distance below which is a good match.

    Returns:
      matches - Lx3 numpy array, of L matches, where L <= N and each column i is
                a match of two descriptors, d_i in image 1 and d_j' in image 2:
                [d_i index, d_j' index, match_score]
    """
    # Check if descriptor dimensions match
    assert desc1.shape[1] == desc2.shape[1]

    # Return zero matches, if one image does not have a keypoint and
    # therefore no descriptors.
    if desc1.shape[0] == 0 or desc2.shape[0] == 0:
        return np.zeros((0, 3))
    if nn_thresh < 0.0:
        raise ValueError('\'nn_thresh\' should be non-negative')

    # Compute L2 distance. Easy since vectors are unit normalized.
    dmat = np.dot(desc1, desc2.T)
    dmat = np.sqrt(2-2*np.clip(dmat, -1, 1))

    # Get NN indices and scores.
    idx = np.argmin(dmat, axis=1)
    scores = dmat[np.arange(dmat.shape[0]), idx]
    
    # Threshold the NN matches.
    keep = scores < nn_thresh
   
    # Check if nearest neighbor goes both directions and keep those.
    idx2 = np.argmin(dmat, axis=0)
    keep_bi = np.arange(len(idx)) == idx2[idx]
    keep = np.logical_and(keep, keep_bi)
    idx = idx[keep]
    scores = scores[keep]
   
    # Get the surviving point indices.
    m_idx1 = np.arange(desc1.shape[0])[keep]
    m_idx2 = idx
    
    # Populate the final Nx3 match data structure.
    matches = np.zeros((int(keep.sum()), 3))
    matches[:, 0] = m_idx1
    matches[:, 1] = m_idx2
    matches[:, 2] = scores
    return matches

def compute_distances_kpts_to_epilines(points_i, points_j, F:np.array) -> np.array:
    """Given two sets of matching points [Nx2] returns the an array of absolute
    distances [Nx2], where as the first column contains the distances of the 
    first points to the  epipolar lines in the first image and the second image
    vise versa."""
    assert points_i.shape[1] == 2
    assert points_j.shape[1] == 2
    assert points_i.shape == points_j.shape
    
    if F is None:
        return (np.zeros((points_i.shape[0], 2)) + np.inf)

    # Epipolar lines in image I of the points in image J
    lines_i = cv2.computeCorrespondEpilines(points_j.reshape(-1, 1, 2), 2, F).reshape(-1, 3)

    # Epipolar lines in image J of the points in image I
    lines_j = cv2.computeCorrespondEpilines(points_i.reshape(-1, 1, 2), 1, F).reshape(-1, 3)

    dist = []
    for k in range(points_i.shape[0]):
        # Params for image i
        xi, yi = points_i[k]
        ai, bi, ci = lines_i[k]

        # Params for image j
        xj, yj = points_j[k]
        aj, bj, cj = lines_j[k]

        di = np.abs(ai*xi + bi*yi + ci) / np.sqrt(ai*ai + bi*bi)
        dj = np.abs(aj*xj + bj*yj + cj) / np.sqrt(aj*aj + bj*bj)

        dist.append((di, dj))

    dist = np.array(dist)
    return dist

def save_stats(
    path_output:str,
    fout_name,
    collection_name:str,
    df:pd.DataFrame,
    fast_eval:bool=False) -> None:
    
    if not os.path.exists(path_output):
        os.makedirs(path_output, exist_ok=True)

    df.to_csv(os.path.join(path_output, fout_name), 
              index=False, 
              encoding='utf-8')

#####################
### DATAFRAME
#####################
"""
collection_name:str             Name of the collection
set_name:str                    Name of the set
kpts_threshold:int              Number of used features
descriptor_name:str             Name of descriptor
detector_name:str               Name of detector
matching_method:str             Name of matching method
desc_distance_threshold:float   Maximal distance of two descriptor to match for nn2w method
ransac_threshold:int            Maximal distance in pixel between two points after estimation
                                of F and projecting the one point to the other.
ransac_confidence:float         Confidence level of the ransac algorithm when estimating
                                the fundamental matrix. [0, 1].

max_num_matches:int             Maximal number of possible matches
num_matches:int                 Actual number of matches
matchability:float              Ratio of num_matches and max_num_matches
accuracy_matches:float          Mean of 1 - (score of match / desc_distance_threshold).
                                Lies in range [0, 1].
mse_matching:float              Mean squared error of matched desriptors.

max_num_inliers:int             Maximal number of inliers for F-matrix
num_inliers:int                 Actual number of inliers for F-matrix
inlier_ratio:float              Ratio between num_inliers and max_num_inliers
accuracy_inliers:               
avg_distance:float              Mean distance between keypoints and corresponding epipolar line
mse_estimation:float            Mean squared error
"""

column_names = ['collection_name', 'set_name', 'kpts_threshold',
                'descriptor_name', 'detector_name', 'matching_method',
                'desc_distance_threshold', 'ransac_threshold',
                'ransac_confidence', 'max_num_matches',
                'num_matches', 'matchability', 'accuracy_matches',
                'mse_matching', 'max_num_inliers', 'num_inliers', 
                'inlier_ratio', 'avg_distance','mse_estimation']

df = pd.DataFrame(columns=column_names)

#####################
### SETTINGS
#####################

root_dir = '/home/mizzade/Workspace/diplom/code'
image_dir = os.path.join(root_dir, 'data')
data_dir = os.path.join(root_dir, 'outputs')
output_dir = '/home/mizzade/Workspace/diplom/outputs/eval_matching_pipeline'

iname1 = '1.png'
iname2 = '2.png'
fname1 = '1_10000.csv'
fname2 = '2_10000.csv'
file_scheme = '_10000.csv'

collection_name = 'eisert'
collection_path_data = os.path.join(data_dir, collection_name)
collection_path_img = os.path.join(image_dir, collection_name)

kpts_thresholds = [1000, 5000, 10000]
desc_distance_thresholds = [0.7]
   
fout_name = 'descriptor_matching_{}_nn2w_fmatrix'.format(collection_name)

ransac_threshold = 3
ransac_confidence = .99

#####################
### MAIN
#####################
# NOTE: To avoid memory errors, handle number of descriptors, detecotrs,
# keypoint threshold etc with car.


set_names = sorted([x for x in os.listdir(collection_path_img) \
                    if os.path.isdir(os.path.join(collection_path_img, x))])

fast_eval = False
verbose = False

fout_name = fout_name + '_fast.csv' if fast_eval else fout_name + '.csv'

for set_name in tqdm(set_names):
    if verbose:
        print(set_name)
    
    # 1. Open folder of set
    set_path_2_desc = os.path.join(collection_path_data, set_name, 'descriptors')
    desc_names = sorted([x for x in os.listdir(set_path_2_desc) \
                               if os.path.isdir(os.path.join(set_path_2_desc, x))])

    descriptor_names = desc_names[:2] if fast_eval else desc_names
    for descriptor_name in descriptor_names:
        if verbose:
            print(descriptor_name)
        
        set_path_2_dets = os.path.join(set_path_2_desc, descriptor_name)
        
        det_names = sorted([x for x in os.listdir(set_path_2_dets) \
                                 if os.path.isdir(os.path.join(set_path_2_dets, x))])
        
        detector_names = det_names[:2] if fast_eval else det_names
        for detector_name in detector_names:
            if verbose:
                print('\t', detector_name)
            
            set_path_2_files = os.path.join(set_path_2_dets, detector_name)
            file_names = sorted([x for x in os.listdir(set_path_2_files) if file_scheme in x])
            
            # 2. Open detector keypoints.
            kpts_path = os.path.join(collection_path_data, set_name, 'keypoints', detector_name)
            kpts1 = pd.read_csv(os.path.join(kpts_path, fname1), sep=',', comment='#', header=None, usecols=[0, 1]).values
            kpts2 = pd.read_csv(os.path.join(kpts_path, fname2), sep=',', comment='#', header=None, usecols=[0, 1]).values

            # 3. Open corresponding descriptors
            desc_path = os.path.join(collection_path_data, set_name, 'descriptors', descriptor_name, detector_name)
            desc1 = pd.read_csv(os.path.join(desc_path, fname1), sep=',', comment='#', header=None).values
            desc2 = pd.read_csv(os.path.join(desc_path, fname2), sep=',', comment='#', header=None).values
            desc1 = normalize_descriptors(desc1)
            desc2 = normalize_descriptors(desc2)
            
            kpts_thresholds = kpts_thresholds[:1] if fast_eval else kpts_thresholds
            for kpts_thresh in kpts_thresholds:
                # 4. Get the subset of descriptors and detectors
                # Make a copy, otherwise you overwrite the slices of 
                # the original.
                d1 = desc1[:kpts_thresh].copy()
                d2 = desc2[:kpts_thresh].copy()
                k1 = kpts1[:kpts_thresh].copy()
                k2 = kpts2[:kpts_thresh].copy()
                
                max_num_matches = np.min([len(d1), len(d2)])
                
                for desc_dist in desc_distance_thresholds:
                    # 5. Make Nearest Neighbour Two Way Matching for each desc_dist
                    # Column1 contains match indices of d1.
                    # Column2 contains match indices of d2.
                    # Column3 contains distance of descriptors.
                    res = nn_match_two_way(d1, d2, desc_dist)
                    
                    num_matches = len(res)
                    matchability = 0 \
                        if max_num_matches == 0 \
                        else num_matches / max_num_matches
                    
                    accuracy_matches = 0 \
                        if num_matches == 0 \
                        else 1.0 - np.mean(res[:, 2] / desc_dist)
                    
                    mse_matching = 0 \
                        if num_matches == 0 \
                        else np.mean(np.linalg.norm(res[:, 2]))
                    
                    max_num_inliers = len(res)
                    
                    # Indices of kpts1 and kpts2 matches
                    idx1 = res[:, 0].astype(np.int)
                    idx2 = res[:, 1].astype(np.int)
                    
                    # Get matching keypoints. Float32 for using cv2 functions.
                    hits1 = (k1.copy()[idx1]).astype(np.float32)
                    hits2 = (k2.copy()[idx2]).astype(np.float32)
                    
                    # ransacReprojThreshold: Maximal distance in pixels from point to epipolar line
                    # to be condiered inlier.
                    # confidence: Confidence value that fundamental matrix is correct.
                    F, mask = cv2.findFundamentalMat(
                        hits1, 
                        hits2, 
                        method=cv2.FM_RANSAC,
                        ransacReprojThreshold=ransac_threshold,
                        confidence=ransac_confidence)
                    
                    if  mask is not None:
                        num_inliers = 0 if mask is None else np.sum(mask)
                        inlier_ratio = 0 if len(res) == 0 else np.sum(mask) / len(res)
                        
                        mask = mask.ravel()
                        hits1 = hits1[mask==1]
                        hits2 = hits2[mask==1]

                        # Absolute distances of points to corresponding epipoloar line
                        # Column1: distances of points in image1 to epilines
                        # Column2: distances of points in image2 to epilines
                        #d_errors = compute_distances_kpts_to_epilines(hits1, hits2, F)
                        d_errors = compute_distances_kpts_to_epilines(hits1, hits2, F)
                        d_errors = d_errors.reshape(-1, 1)
                        
                        mean_dist = np.mean(d_errors)
                        
                        accuracy_inliers = 0 \
                            if num_inliers == 0 \
                            else 1.0 - mean_dist / ransac_threshold
                        
                        mse_estimation = 0 \
                            if len(d_errors) == 0 \
                            else np.linalg.norm(d_errors) / len(d_errors)

                    else:
                        num_inliers = 0
                        inlier_ratio = 0
                        mean_dist = np.nan
                        mse_estimation = np.nan
                        accuracy_inliers = 0
                        
                    # Add to dataframe
                    df = df.append({
                        'collection_name': collection_name, 
                        'set_name': set_name, 
                        'kpts_threshold': kpts_thresh,
                        'descriptor_name': descriptor_name, 
                        'detector_name': detector_name, 
                        'matching_method': 'nn2w',
                        'desc_distance_threshold': desc_dist,
                        'ransac_threshold': ransac_threshold,
                        'ransac_confidence': ransac_confidence,
                        'max_num_matches': max_num_matches,
                        'num_matches': num_matches,
                        'matchability': matchability,
                        'accuracy_matches': accuracy_matches,
                        'mse_matching': mse_matching,
                        'max_num_inliers': max_num_inliers,
                        'num_inliers': num_inliers,
                        'inlier_ratio': inlier_ratio,
                        'accuracy_inliers': accuracy_inliers,
                        'avg_distance': mean_dist,
                        'mse_estimation': mse_estimation
                    }, ignore_index=True)
                    
                    
                    # Free memory
                    del idx1, idx2, F, mask, num_inliers, inlier_ratio, mean_dist, mse_estimation, hits1, hits2
                    del num_matches, matchability, accuracy_matches, mse_matching, accuracy_inliers
                    gc.collect()
                
                # Free memory
                del d1, d2, k1, k2, max_num_matches
                gc.collect()
            
            # Free memory
            del kpts1, kpts2, desc1, desc2
            gc.collect()

save_stats(
    output_dir,
    fout_name,
    collection_name,
    df,
    fast_eval=fast_eval)